<a href="https://colab.research.google.com/github/medikid/ai_lotto/blob/master/notebooks/git_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Convert IPyNB to Py

In [ ]:
#Save all ipynb files to py and make sure first line has %%WRITEFILE to py files

%run 'ifile.ipynb'

%run 'idata.ipynb'
%run 'idataset.ipynb'

%run 'imodel.ipynb'
%run 'ker_model.ipynb'
%run 'ker_model_loader.ipynb'
%run 'tfl_model.ipynb'


%run 'ai.ipynb'
%run 'ipredictor.ipynb'
%run 'itrainer.ipynb'

## Run Py files to memory

In [ ]:
# %load '../load_lib.py'
# then import lib as 'from ifile import iFile' on your notebooks

%run '../ifile.py'
%run '../idata.py'
%run '../idataset.py'
%run '../imodel.py'
%run '../ker_model.py'
%run '../ker_model_loader.py'
%run '../tfl_model.py'
%run '../ai.py'
%run '../itrainer.py'


# GIthub Functions

## Git ADD, COMMIT and PUSH to remote

In [ ]:
!git add -A .
!git commit -m "Added git functions"
!git push origin master

## Git Pull from remote github repo

In [5]:
!git pull

Already up to date.


## Git conflict local - so reset local to match remote

In [ ]:
!git fetch --all
!git reset --hard origin/master
!git pull

## Unpushed local changes - Stash local, Pull Remote and write local changes to pulled remote

In [ ]:
!git stash
!git pull
!git stash pop

## Setup git config repo

In [ ]:
#!git config --global user.email "soma.programmer@gmail.com"
#!git config --global user.name "medikid"

#!git remote set-url origin https://github.com/medikid/ai_lotto.git


In [ ]:
!git remote show origin

# Sync G-Drive with Data Folder

In [ ]:
#!pip install dirsync

from dirsync import sync
source_path = '../g_drive'
target_path = '../data'

sync(source_path, target_path, 'sync', verbose  = True) #for syncing one way
sync(target_path, source_path, 'sync', verbose = True) #for syncing the opposite way

# sync(source_path, target_path, 'diff') #for syncing one way
# sync(target_path, source_path, 'diff') #for syncing the opposite way

In [ ]:
!tree '../data/'

In [ ]:
import os, shutil

#shutil.copy(a,b)
src_folder = "../data"
dst_folder = "../archive"
for src_file_path in [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser("../data")) for f in fn]:
    src_folder_path = os.path.dirname(src_file_path)
    src_file_format = src_file_path.split('.')[-1]; # 'h5'
    src_file_name = os.path.basename(src_file_path);
    game = st.split('/')[2]
#     print("Full path: {0} \n \
#     folder_path: {1} \n \
#     file_name: {2} \n \
#     file_format: {3}".format(full_path, folder_path, file_name, file_format ));
    if (src_file_format == 'h5'):
        dst_file_name = '[{0}]{1}'.format(src_dataset_id, src_file_name);
        dst_file_path = '{0}/{1}/models/{2}.{3}'.format(dst_folder, game, dst_file_name,src_file_format)

        
    

In [ ]:
st = '../data/keno/master/keno_master_dr'
#st.split('.')[-1]
st.split('/')[2]

## Archive files into archive folder

In [1]:
%%writefile '../archiver.py'

import os, shutil

class Archiver:
    srcFolder = "../data"
    dstFolder = "../archive"
    srcFiles = []
    dstFiles = []
    
    def __init__(self, src='../data', dst='../archive'):
        self.setup(src, dst);
        self.process_files();
        
        self.setup('../g_drive', dst);
        self.process_files();
        
    def setup(self, src, dst):
        self.srcFolder = src;
        self.dstFolder = dst;
        self.srcFiles = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(self.srcFolder)) for f in fn];
        self.dstFiles = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(self.dstFolder)) for f in fn];
        print('Source:{0} Files'.format(len(self.srcFiles)))
        print('Dest: {0} Files'.format(len(self.dstFiles)))
        
        
    def process_files(self):        
        for srcFilePath in self.srcFiles:
            srcFolderPath = os.path.dirname(srcFilePath)
            srcFileFormat = srcFilePath.split('.')[-1]; # 'h5'
            srcFileName = os.path.basename(srcFilePath);
            game = srcFilePath.split('/')[2]
            if (srcFileFormat == 'h5'): 
                pass;
                self.archive_h5(srcFilePath);
            elif (srcFileFormat == 'pkl'):
                pass;
                self.archive_pkl(srcFilePath);
            elif (srcFileFormat == 'npz'):
                self.archive_npz(srcFilePath);
                print('FileName:{0}[{1} MB]'.format(srcFilePath,round(os.path.getsize(srcFilePath)/(1024*1024)),2) )
            else:
                print("Unknown format: {0}".format(srcFilePath))
                    
    def archive_h5(self, srcFilePath):
        srcFolderPath = os.path.dirname(srcFilePath)
        srcFileName = os.path.basename(srcFilePath);
        srcFileFormat = srcFilePath.split('.')[-1];
        folders = srcFolderPath.split('/');
        try:
            game = folders[2];
            xnInputs = folders[3];
            xnDraws = folders[4]
            srcDatasetID = '{0}_{1}_{2}_dr'.format(game,xnInputs,xnDraws)
            dstFileName = '[{0}]{1}'.format(srcDatasetID, srcFileName);
            dstFilePath = '{0}/{1}/models/{2}'.format(self.dstFolder, game, dstFileName)
            #print('{0}=>{1}'.format(srcFilePath, dstFilePath))
            if(os.path.exists(dstFilePath)):
                pass;
            else:
                self.archive(srcFilePath, dstFilePath);
            
        except IndexError:
            if (folders[3] == 'untrained_models'):
                game = folders[2]
                dstFileName = '[untrained]{0}'.format(srcFileName);
                dstFilePath = '{0}/{1}/models/{2}'.format(self.dstFolder, game, dstFileName)
                if(os.path.exists(dstFilePath)):
                    pass;
                else:
                    self.archive(srcFilePath, dstFilePath);
            else:
                print("exception",srcFilePath)
    
    def archive_pkl(self, srcFilePath):
        game = srcFilePath.split('/')[2]
        isMaster = srcFilePath.split('/')[3]
        srcFileName = os.path.basename(srcFilePath);
        dstFileName = srcFileName;
        if (isMaster == 'master'):
            dstFilePath = '{0}/{1}/masters/{2}'.format(self.dstFolder, game, dstFileName)
        
        if(os.path.exists(dstFilePath)):
                pass;
        else:
            self.archive(srcFilePath, dstFilePath);
            
    def archive_npz(self, srcFilePath):
        game = srcFilePath.split('/')[2]
        srcFileName = os.path.basename(srcFilePath);
        dstFileName = srcFileName;
        dstFilePath = '{0}/{1}/datasets/{2}'.format(self.dstFolder, game, dstFileName)
        #print(srcFilePath)
        if(os.path.exists(dstFilePath)):
                pass;
        else:
            self.archive(srcFilePath, dstFilePath);
            
    def archive(self, srcFilePath, dstFilePath):
        shutil.copy(srcFilePath, dstFilePath);
        print("Copying {0}".format(os.path.basename(dstFilePath)));
    

Overwriting ../archiver.py


In [2]:
%run '../archiver.py'
arc = Archiver('../data');

Source:170 Files
Dest: 477 Files
Unknown format: ../data/print_folder_tree.ipynb
FileName:../data/hotspot/x15/25k/datasets/hotspot_x15_25k_dp.npz[141 MB]
FileName:../data/hotspot/x15/25k/datasets/hotspot_x15_25k_dP.npz[0 MB]
FileName:../data/hotspot/x15/25k/datasets/hotspot_x15_25k_dr.npz[244 MB]
FileName:../data/hotspot/x15/300k/datasets/hotspot_x15_300k_dr.npz[2932 MB]
FileName:../data/hotspot/x15/300k/datasets/hotspot_x15_300k_dp.npz[0 MB]
FileName:../data/hotspot/x15/200k/datasets/hotspot_x15_200k_dp.npz[0 MB]
FileName:../data/hotspot/x15/200k/datasets/hotspot_x15_200k_dr.npz[1955 MB]
Copying [hotspot_x15_310k_dr]hotspot.ker.4.2.e0370.h5
Copying [hotspot_x15_310k_dr]hotspot.ker.4.2.e0375.h5
Copying [hotspot_x15_310k_dr]hotspot.ker.4.2.e0380.h5
Copying [hotspot_x15_310k_dr]hotspot.ker.4.3.e0001.h5
FileName:../data/hotspot/x15/310k/datasets/hotspot_x15_310k_dp.npz[0 MB]
FileName:../data/hotspot/x15/310k/datasets/hotspot_x15_310k_dr.npz[3030 MB]
FileName:../data/hotspot/x15/100k/datas

# Print folder tree

In [4]:
!tree '../archive/keno'

../archive/keno
├── datasets
│   ├── keno_x15_100k_dr.npz
│   ├── keno_x15_1m_dr.npz
│   ├── keno_x15_250k_dr.npz
│   ├── keno_x15_2.528m_dr.npz
│   ├── keno_x15_25k_dr.npz
│   ├── keno_x15_2m_dr.npz
│   └── keno_x15_500k_dr.npz
├── masters
│   ├── keno_master_dpf.pkl
│   ├── keno_master_dp.pkl
│   ├── keno_master_drf.pkl
│   ├── keno_master_dr.pkl
│   ├── keno_master.pkl
│   └── keno_master_r.pkl
└── models
    ├── [keno_x15_2.528m_dr]keno.ker.1.1.h5
    ├── [keno_x15_2.528m_dr]keno.ker.2.1.h5
    ├── [keno_x15_2.528m_dr]keno.ker.4.1.e0001.h5
    ├── [keno_x15_2.528m_dr]keno.ker.4.1.e0002.h5
    ├── [keno_x15_2.528m_dr]keno.ker.4.1.e0003.h5
    ├── [keno_x15_2.528m_dr]keno.ker.4.1.e0004.h5
    ├── [keno_x15_2.528m_dr]keno.ker.4.1.e0005.h5
    ├── [keno_x15_2.528m_dr]keno.ker.4.1.e0006.h5
    ├── [keno_x15_2.528m_dr]keno.ker.4.1.e0007.h5
    ├── [keno_x15_2.528m_dr]keno.ker.4.1.e0008.h5
    ├── [keno_x15_2.528m_dr]keno.ker.4.1.e0009.h5
    ├── [keno_x15_2.528m_dr]keno.ker.4.1.e0010.h5
